# <font color=orange>**OpenCV ile Nesne Takibi**</font>

## <font color=gold>**Ortalama Kayma Algoritmasi**</font> <font color=gray>**(Meanshift Camshift)**</font>

* ***Meanshift (Ortalama Kayma),*** noktalari Mod'a dogru kaydirarak veri noktalarini kumelere yinelemeli olarak atayan bir kumeleme algoritmasidir.

* Mod, en yuksek veri noktasi yogunlugu olarak tanimlanabilir.
* Bu nedenle, <font color=red>**Mod arama**</font> algoritmasi olarak da bilinir.

In [7]:
import cv2
import numpy as np

# Kamera acma:
cap = cv2.VideoCapture(0)

# Bir tane frame oku:
ret, frame = cap.read()

if ret == False: # Kamera acildiginda yuz goremezse hata verirken uyari versin.
    print('Uyarı!!!')

# Detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)
#kamera acar acmaz bir yuz dogrultmak gereklidir.

(face_x, face_y, w, h) = tuple(face_rects[0])

# meanshift algoritmasi girdisi
track_windows = (face_x, face_y, w, h)

# Region of interest
roi = frame[face_y:face_y + h, face_x:face_x + w] # Roi = face

# Calisma hsv formatinda incelenecek:
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Histogram: Takip icin gereklidir.
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Normalize:
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Takip icin gerekli durdurma kriterleri:
# count : Hesaplanacak maksimum oge sayisi
# epsilon: degisiklik
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1) #yani 5 yineleme veya 1 epsilon

while True:
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        '''
        - Bir ozelligin hist hesaplanir ve ardindan ozelligi bir govde de bulmak icin kullanir 
        - Histogrami bir goruntude bulmak icin kullanilir.
        - Belirli bir goruntunun piksellerinin bir hist. modelindeki piksel dagilimina ne kadar
          uydugunu kaydetmenin bir yoludur. (Piksel Karsilastrma) 
        '''
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1)
        
        
        # Meanshift kullanimi:
        ret, track_windows = cv2.meanShift(dst, track_windows, term_crit)
        x,y,w,h = track_windows
        
        img2 = cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 5)
        cv2.imshow('Takip', img2)
        
        if cv2.waitKey(1) & 0xFF == ord('q'): break
        
cap.release()
cv2.destroyAllWindows()